<a href="https://colab.research.google.com/github/TiredRebel/Math_hw_1/blob/main/4_Trend_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

# Дані
t = np.array([1, 2, 3, 4, 5], dtype=float)
y = np.array([22, 28, 37, 45, 53], dtype=float)

# 1) Формуємо матрицю A: [t, 1]
A = np.column_stack([t, np.ones_like(t)])

# 2) МНК через lstsq: x = [k, b]
x_lstsq, residuals, rank, s = np.linalg.lstsq(A, y, rcond=None)
k_lstsq, b_lstsq = x_lstsq

print("=== np.linalg.lstsq ===")
print("k =", k_lstsq)
print("b =", b_lstsq)

# 3) Прогноз на 6-ту годину
y6 = k_lstsq * 6 + b_lstsq
print("y_hat(6) =", y6)

# 4) Опціонально: нормальні рівняння A^T A x = A^T y
ATA = A.T @ A
ATy = A.T @ y
x_normal = np.linalg.solve(ATA, ATy)
k_n, b_n = x_normal

print("\n=== Normal equations ===")
print("A^T A =\n", ATA)
print("A^T y =", ATy)
print("k =", k_n)
print("b =", b_n)

# 5) Перевірка збігу результатів
print("\nDifference (lstsq - normal):", x_lstsq - x_normal)


=== np.linalg.lstsq ===
k = 7.9
b = 13.299999999999995
y_hat(6) = 60.7

=== Normal equations ===
A^T A =
 [[55. 15.]
 [15.  5.]]
A^T y = [634. 185.]
k = 7.9
b = 13.300000000000002

Difference (lstsq - normal): [ 0.00000000e+00 -7.10542736e-15]


## Задача 4. Прогноз тренду (МНК)

**Коментар:** Дані навантаження містять шум, але видно зростаючий тренд.  
Побудуємо лінію тренду (найкраще наближення) у вигляді:

$$
\hat y = kt + b
$$

Оскільки точки не лежать ідеально на прямій, точного розв’язку не існує — використовуємо **метод найменших квадратів (МНК)**.

---

### Дано

Час вимірювань:

$$
t = [1,2,3,4,5]
$$

Навантаження CPU:

$$
y = [22,28,37,45,53]
$$

---

## 1) Формуємо матрицю $A$

**Коментар:** Щоб знайти $k$ і $b$, створюємо матрицю $A$ з двох стовпців:  
перший — значення $t$, другий — одиниці (для вільного члена $b$).

$$
A=
\begin{pmatrix}
1 & 1\\
2 & 1\\
3 & 1\\
4 & 1\\
5 & 1
\end{pmatrix},
\qquad
\mathbf{y}=
\begin{pmatrix}
22\\
28\\
37\\
45\\
53
\end{pmatrix},
\qquad
\mathbf{x}=
\begin{pmatrix}
k\\
b
\end{pmatrix}
$$

МНК-розв’язок можна знаходити через `lstsq`, а також через **нормальні рівняння**:

$$
A^T A\,\mathbf{x} = A^T \mathbf{y}
$$

---

## 2) Аналітичне знаходження $k$ і $b$ (через формули МНК)

**Коментар:** Для прямої $\hat y = kt + b$ зручно використати формули через суми:

$$
k=\frac{n\sum (t_i y_i) - (\sum t_i)(\sum y_i)}{n\sum (t_i^2) - (\sum t_i)^2},
\qquad
b=\frac{\sum y_i - k\sum t_i}{n}
$$

Обчислюємо потрібні суми:

$$
n=5,\qquad \sum t = 1+2+3+4+5 = 15
$$

$$
\sum t^2 = 1^2+2^2+3^2+4^2+5^2 = 1+4+9+16+25 = 55
$$

$$
\sum y = 22+28+37+45+53 = 185
$$

$$
\sum (ty)=1\cdot22+2\cdot28+3\cdot37+4\cdot45+5\cdot53
=22+56+111+180+265=634
$$

### Знаходимо $k$

$$
k=\frac{5\cdot634 - 15\cdot185}{5\cdot55 - 15^2}
=\frac{3170-2775}{275-225}
=\frac{395}{50}=7.9
$$

### Знаходимо $b$

$$
b=\frac{185 - 7.9\cdot15}{5}
=\frac{185-118.5}{5}
=\frac{66.5}{5}=13.3
$$

Отже, рівняння тренду:

$$
\hat y = 7.9t + 13.3
$$

---

## 3) Прогноз на 6-ту годину

**Коментар:** Підставляємо $t=6$ у трендову пряму.

$$
\hat y(6)=7.9\cdot 6 + 13.3 = 47.4 + 13.3 = 60.7
$$

---

## 4) Опціонально: розв’язок через нормальні рівняння $A^T A\mathbf{x}=A^T\mathbf{y}$

**Коментар:** Побудуємо $A^TA$ та $A^T\mathbf{y}$, а потім розв’яжемо систему для $\mathbf{x}=[k,b]^T$.

Оскільки

$$
A=
\begin{pmatrix}
t_1 & 1\\
t_2 & 1\\
t_3 & 1\\
t_4 & 1\\
t_5 & 1
\end{pmatrix},
$$

то маємо стандартні суми:

$$
A^TA=
\begin{pmatrix}
\sum t^2 & \sum t\\
\sum t & n
\end{pmatrix}
=
\begin{pmatrix}
55 & 15\\
15 & 5
\end{pmatrix}
$$

$$
A^T\mathbf{y}=
\begin{pmatrix}
\sum (ty)\\
\sum y
\end{pmatrix}
=
\begin{pmatrix}
634\\
185
\end{pmatrix}
$$

Отже, нормальні рівняння:

$$
\begin{pmatrix}
55 & 15\\
15 & 5
\end{pmatrix}
\begin{pmatrix}
k\\
b
\end{pmatrix}
=
\begin{pmatrix}
634\\
185
\end{pmatrix}
$$

Розв’язок (має збігатися з `lstsq`):

$$
k=7.9,\qquad b=13.3
$$

---

## Відповідь

- Рівняння тренду: $$\hat y = 7.9t + 13.3$$
- Прогноз на 6-ту годину: $$\hat y(6)=60.7$$
